In [3]:
from pycocotools.coco import COCO
import cv2
import os
from tqdm import tqdm

#Converts taco annotations from coco to yolo format
taco_p = r"C:\Users\Computing\OneDrive - University of Lincoln (1)\Documents\TACO\TACO\data"

def getTaco(p):   
    ann_path = os.path.join(p, "annotations.json")
    coco = COCO(ann_path)
    return coco

In [4]:
t = getTaco(taco_p)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [32]:
ids = t.getImgIds()
dest = r"C:\Users\Computing\OneDrive - University of Lincoln (1)\Documents\TACO_YOLO"

for imId in tqdm(ids): #for each image selected
    #get image object from coco annotations
    img = t.imgs[imId]
        
    #get file name and join it to data path
    imp = os.path.join(taco_p, img['file_name'])
  

    #get new name for saving new image
    new_im_name = img['file_name'].split("/")[0] + "_" + img['file_name'].split("/")[1].split(".")[0] + ".jpg"
    new_im_path = os.path.join(dest,"images")
    new_im_path = os.path.join(new_im_path,new_im_name)
    
    #get image
    gv_im = cv2.imread(imp)
    
    #get image width and height
    width = img['width']
    height = img['height']
    
    #get category ids
    cat_ids = t.getCatIds()
    
    #get annotation ids
    anns_ids = t.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
    anns = t.loadAnns(anns_ids)


    #for each annotation....
    try:
        cv2.imwrite(new_im_path, gv_im)
    except:
        print(img['file_name'], "not found, skipping save")
        continue
        
    for i, ann in enumerate (anns):
        #get bounding box
        [x,y,w,h] = anns[i]['bbox']
        
        #convert bounding box to yolo format

        #save to file
        xcen = float(x + (w/2)) / width #normalise bboxes for yolo according to image size (YOLO is in format <class> <x centroid> <y centroid> <bb width> <bb height>)
        ycen = float(y + (h/2)) / height

        w = float(w) / width
        h = float(h) / height

        im_name = new_im_name.split('.')[:-1]
        im_name.append('txt') #get image name from text name
        ann_Txt = ".".join(im_name)
        dest_labels = os.path.join(dest,"labels") #save txt file as image name with extension change
        dest_save = os.path.join(dest_labels,ann_Txt)
        
        with open(dest_save, 'a+') as f:
            f.write(f"0 {xcen} {ycen} {w} {h}\n")



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [08:13<00:00,  3.04it/s]
